In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [3]:
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
base = 'POC-look-alike-ds-nac'
prefix = 'sagemaker/' + base

role = sagemaker.get_execution_role()

In [1]:
!mkdir /tmp/events/
!aws s3 cp s3://poc-la-ds-nac/data/events/2019-09-12/events-email-hashed.csv /tmp/events/

download: s3://poc-la-ds-nac/data/events/2019-09-12/events-email-hashed.csv to ../../../../tmp/events/events-email-hashed.csv


In [38]:
import csv
count = 0
with open("/tmp/events/events-email-hashed.csv", "r") as raw_csv:
    csv_data = csv.reader(raw_csv)
    with open("/tmp/events/products-only-events.csv", "w") as filtered_csv:
        header = None
        csv_filtered = csv.writer(filtered_csv)
        for l in csv_data:
            if not header:
                header = l
                header[0:0] = ["realProfileId"]
                csv_filtered.writerow(header)
                continue
            if not l[9]:
                continue
            csv_filtered.writerow(l)
            count = count + 1
print("Done. {} lines".format(count))

                    

Done. 711981 lines


In [45]:
ds_events = pd.read_csv("/tmp/events/products-only-events.csv", header='infer', error_bad_lines=False)
print(len(ds_events.realProfileId.unique()))
ds_events.head()

550905


,realProfileId,profileId,eventType,userId,eventId,timestamp,timezone,productAmount,productCost,productId,...,email,productPrice,productName,deviceType,os,deviceMaker,browserName,deviceModel,interactionTypeId,referer
0,000024c91ce3fc0e9596f37bba7bd47e,NaN,purchase,4852a9c8-2408-425a-9806-69ad2f0e1e23,5ecf41e1-1d83-4b44-a2a8-4fd6980f451c,2019-02-14T11:37:50.261,UTC,2.0,134.5,18666,...,55f3ec874cecb4459dda1ce7be2bc07f,67.25,Farruquito,NaN,iOS,NaN,NaN,NaN,NaN,NaN
1,000024c91ce3fc0e9596f37bba7bd47e,NaN,purchase,7cef779d-fe5d-4500-ba2a-a669439e6cf1,8c48de80-0335-4457-8eb9-c1b2cdddb5f8,2018-04-09T11:13:15.528,UTC,2.0,798.0,19146,...,55f3ec874cecb4459dda1ce7be2bc07f,399.00,NAC Gala CNA- Diana Ross,NaN,iOS,NaN,NaN,NaN,NaN,NaN
2,0000a743728ace7657f01fda8617d936,NaN,purchase,220cf84b-78c4-44f1-9126-844e3b7ea025,d0caa392-b575-4995-8431-6a6a0070ef15,2019-09-10T12:35:55.406,UTC,2.0,106.0,21656,...,74ad1b30de99a25d17ca9d971dbe4449,53.00,J'aime Hydro,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000a743728ace7657f01fda8617d936,NaN,purchase,a7b9c42a-347c-413f-8c95-9c9ecc300e72,1704797b-1b1e-4932-8ef2-b9d8fbd97c2a,2018-09-03T08:28:35.252,UTC,2.0,60.5,18634,...,74ad1b30de99a25d17ca9d971dbe4449,30.25,Le tigre bleu de l'Euphrate,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000a743728ace7657f01fda8617d936,NaN,purchase,3b30dd13-3696-4817-8b40-f6580d91df8a,13b6b478-9779-4e9c-b2b4-8f56c708b7bd,2018-08-21T21:12:35.886,UTC,2.0,115.0,17760,...,74ad1b30de99a25d17ca9d971dbe4449,57.50,Notre Dame de Paris,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
print("Emails amount: {0}".format(len(ds_events.email.unique())))
print("Products amount: {0}".format(len(ds_events.productId.unique())))
ds_product_view = ds_events[ds_events["eventType"] == "product_view"]
print("Product view amount: {0}".format(len(ds_product_view)))
product_view_unique_ids = ds_product_view.realProfileId.unique()
print("Product view unique user ids: {0}".format(len(product_view_unique_ids)))
ds_purchases = ds_events[ds_events["eventType"] == "purchase"]
print("Purchases amount: {0}".format(len(ds_purchases)))
purchase_unique_ids = ds_purchases.realProfileId.unique()
print("Purchases with unique user ids: {0}".format(len(purchase_unique_ids)))
print("Unique users with both product view and purchase: {0}".format(len(np.intersect1d(purchase_unique_ids, product_view_unique_ids))))

Emails amount: 504673
Products amount: 1532
Product view amount: 568431
Product view unique user ids: 499745
Purchases amount: 143550
Purchases with unique user ids: 52923
Unique users with both product view and purchase: 1763


In [ ]:
added_events = pd.read_csv("s3://poc-la-ds-nac/data/nac-nightly-upload/2019-05-12/added_events_2019-05-12_12-30-39.csv")
index_events = pd.read_csv("s3://poc-la-ds-nac/data/nac-nightly-upload/2019-05-12/index_events_2019-05-12_12-30-39.csv")
tickets_and_plans = pd.read_csv("s3://poc-la-ds-nac/data/nac-nightly-upload/2019-05-12/tickets_and_plans_2019-05-12_12-30-39.csv")
#email_mods = pd.read_csv("s3://poc-la-ds-nac/data/nac-nightly-upload/email_mods_yesterday_2019-09-16_12-30-40.csv")
#attendance_yesterday = pd.read_csv("s3://poc-la-ds-nac/data/nac-nightly-upload/attendance_yesterday_2019-09-16_12-30-40.csv")
#gift_cards = pd.read_csv("s3://poc-la-ds-nac/data/nac-nightly-upload/index_packages_prod_2018-09-02_12-30-55.csv")

In [28]:
added_events.head(20)

,event_id,event_name,event_name_long,season_id,season_year,arena_name,event_date,event_time,event_day,event_type,team,onsale_datetime,offsale_datetime,tm_event_name,dw_event_id,major_category,minor_category,url
0,7687,D9M0511,Le Patin Libre - Threshold,220,2018,General Admission - Outside,2019-05-11,19:30:00,SAT,Single Event,Le Patin Libre,NaN,NaN,EZ9ME11,310054CF99E81C58,FAMILY,ICE SHOWS,18670.0
1,7688,D9M0511M,Le Patin Libre - Threshold,220,2018,General Admission - Outside,2019-05-11,17:00:00,SAT,Single Event,Le Patin Libre,NaN,NaN,EZ9ME11M,310054CF99E31C4F,FAMILY,ICE SHOWS,18670.0
2,7717,E9S0511M,Between Breaths,211,2018,Studio Gen Adm,2019-05-11,14:00:00,SAT,Single Event,Between Breaths,NaN,NaN,EZ9SE11M,310054C6C1A51CF8,ARTS,THEATRE OTHER,18800.0
3,7718,E9S0511,Between Breaths,211,2018,Studio Gen Adm,2019-05-11,20:00:00,SAT,Single Event,Between Breaths,NaN,NaN,EZ9SE11,310054C6C19F1CF6,ARTS,THEATRE OTHER,18800.0
4,8333,R9T0524L,ILIZA : Elder Millennial,228,2018,Theatre - Th��tre,2019-05-24,21:30:00,FRI,Single Event,ILIZA,2018-11-21 09:00:00.0,2019-05-24 21:30:00.0,EZ9TE24L,310055669CAA312B,CONCERTS,COMEDY,20429.0
5,8550,R9O0511,Alessia Cara,200,2018,Salle Southam Hall,2019-05-11,20:00:00,SAT,Single Event,Alessia Cara,NaN,NaN,EZ9OE11,310056399E312B34,CONCERTS,R & B,20499.0
6,8652,V9F0511,Martyn Joseph,203,2018,Fourth Stage,2019-05-11,20:30:00,SAT,Single Event,Martyn Joseph,NaN,NaN,EZ9XE11,3100564E9989235E,CONCERTS,FOLK,21672.0
7,9054,R9O0609,Cappies Gala,200,2018,Salle Southam Hall,2019-06-09,18:30:00,SUN,Single Event,Cappies Gala,2019-05-15 09:00:00.0,2019-06-09 19:00:00.0,EZ9OF09,310056A79F921976,ARTS,THEATRE OTHER,20244.0
8,9062,N9U0617,Young Artist Program - Precollege YAP - Concert,220,2018,General Admission - Outside,2019-06-17,19:00:00,MON,Single Event,Young Artist Program,2019-05-06 09:00:00.0,2019-06-17 19:00:00.0,EZ9UF17,310056AA90D51F5F,CONCERTS,CLASSICAL/VOCAL,NaN


In [27]:
index_events.head(20)
#e_18800 = index_events[index_events['event_id'] == 18670]
#e_18800.head()

,event_id,event_date,event_date_timestamp,event_date_formatted_en,event_date_formatted_fr,event_name,event_name_fr,event_category,sale_start_date,sale_end_date,...,parent_url,parent_url_fr,performance_url,performance_url_fr,event_description_en,event_description_fr,event_description_excerpt_en,event_description_excerpt_fr,recommendable,is_sold_out
0,18800,5/7/2019 20:00:00,1557273600,"May 7 - 18, 2019",7 - 18 mai 2019,NaN,NaN,Theatre,6/25/2018 10:00:00,5/18/2019 12:00:00,...,https://nac-cna.ca/en/event/18800,https://nac-cna.ca/fr/event/18800,https://www.ticketmaster.ca/artist/2514017?lan...,https://www.ticketmaster.ca/artist/2514017?lan...,<p>An Artistic Fraud of Newfoundland (St. John...,<p>Une production d\Artistic Fraud of Newfound...,<p><em>Between Breaths&nbsp;</em>floats throug...,<p><em>Between Breaths&nbsp;</em>&eacute;voque...,NaN,no
1,18657,5/15/2019 20:00:00,1557964800,"May 15 - 16, 2019",15 - 16 mai 2019,Alberta Ballet,Alberta Ballet,Dance,6/25/2018 10:00:00,5/16/2019 12:00:00,...,https://nac-cna.ca/en/event/18657,https://nac-cna.ca/fr/event/18657,https://www.ticketmaster.ca/artist/927982?lang...,https://www.ticketmaster.ca/artist/927982?lang...,<p>Danced to the soulful music of treasured si...,<p>Dans&eacute; sur la musique inspirante de l...,<p>Danced to the soulful music of treasured si...,<p>Dans&eacute; sur la musique inspirante de l...,NaN,no
2,18740,5/22/2019 20:00:00,1558569600,"May 22, 2019",22 mai 2019,NaN,NaN,Music,6/25/2018 10:00:00,5/22/2019 12:00:00,...,https://nac-cna.ca/en/event/18740,https://nac-cna.ca/fr/event/18740,https://www.ticketmaster.ca/event/310054B7C2F9...,https://www.ticketmaster.ca/event/310054B7C2F9...,<p>The legendary Sir Andrew Davis and the Toro...,<p>Le l&eacute;gendaire Sir Andrew Davis et l\...,<p>Legendary <strong>Sir Andrew Davis</strong>...,<p>Le l&eacute;gendaire <strong>Sir Andrew Dav...,NaN,no
3,21512,5/23/2019 19:30:00,1558654200,"May 23, 2019",23 mai 2019,NaN,NaN,Music,2/5/2019 10:00:00,5/23/2019 12:00:00,...,https://nac-cna.ca/en/event/21512,https://nac-cna.ca/fr/event/21512,https://www.ticketmaster.ca/event/3100563D83C9...,https://www.ticketmaster.ca/event/3100563D83C9...,"<p>Celebrating a quarter century as a band, NA...","<p>Pour son 25&egrave;me annniversaire, CNA pr...",NaN,NaN,NaN,no
4,20193,5/23/2019 20:00:00,1558656000,"May 23, 2019",23 mai 2019,NaN,NaN,Music,10/26/2018 10:00:00,5/23/2019 12:00:00,...,https://nac-cna.ca/en/event/20193,https://nac-cna.ca/fr/event/20193,https://www.ticketmaster.ca/event/31005551E794...,https://www.ticketmaster.ca/event/31005551E794...,"<p>Grammy-winning singer, songwriter, and comp...",<p>Depuis qu\il a re&ccedil;u le Prix JUNO du ...,"<p>Grammy-winning singer, songwriter, and comp...",NaN,NaN,no
5,22128,5/23/2019 20:30:00,1558657800,"May 23, 2019",23 mai 2019,NaN,NaN,Music,3/22/2019 10:00:00,5/23/2019 12:00:00,...,https://nac-cna.ca/en/event/22128,https://nac-cna.ca/fr/event/22128,https://www.ticketmaster.ca/event/310056729FBD...,https://www.ticketmaster.ca/event/310056729FBD...,<p>Rewind to January 2017. Whitney Rose was pr...,<p>Replongeons-nous en janvier 2017. Whitney R...,<p>&ldquo;Whitney Rose is making country music...,<p>&laquo;&thinsp;La musique country de Whitne...,NaN,no
6,22113,5/24/2019 20:00:00,1558742400,"May 24, 2019",24 mai 2019,NaN,NaN,Music,3/19/2019 10:00:00,5/24/2019 12:00:00,...,https://nac-cna.ca/en/event/22113,https://nac-cna.ca/fr/event/22113,https://www.ticketmaster.ca/event/31005668C6D1...,https://www.ticketmaster.ca/event/31005668C6D1...,"<p>Since arriving in Canada in 2003, Brazilian...",<p>Depuis son arriv&eacute;e &agrave; Toronto ...,NaN,NaN,NaN,no
7,22116,5/24/2019 20:30:00,1558744200,"May 24, 2019",24 mai 2019,Sarah Thawer,Sarah Thawer,Music,3/19/2019 10:00:00,5/24/2019 12:00:00,...,https://nac-cna.ca/en/event/22116,https://nac-cna.ca/fr/event/22116,https://www.ticketmaster.ca/event/3100566B969C...,https://www.ticketmaster.ca/event/3100566B969C...,<p>Sarah Thawer is a self-taught drummer based...,

In [15]:
tickets_and_plans.head()

,event_name,section_name,row_name,seat_num,num_seats,last_seat,team,event_date,event_time,event_day,...,tm_event_name,status,ticket_type,cust_name_id,ticket_seq_id,owner_name,retail_acct_num,dw_event_id,cust_email,other_info_1
0,R9O1129,AMPHRD,A,14,2,16,Downchild Blues Band,2019-11-29,20:00:00.000,FRI,...,EZ9OK29,SOLD,Adult,110029,2280455.0,"Fabijanski, Steven",17-59439,310056A6BA133548,sff@sympatico.ca,English
1,R9O0608N,AMPHLG,A,47,2,49,David Foster,2019-06-08,20:00:00.000,SAT,...,EZ9OF08N,SOLD,Adult,493982,2280456.0,"Corriveau, Andre",18-41718,3100564DC1A52D4B,melaniej@rogers.com,English
2,V9F0511,GN,GN1,41,3,43,Martyn Joseph,2019-05-11,20:30:00.000,SAT,...,EZ9XE11,SOLD,Adult,504308,2280461.0,"Sproule, Lynne Dee",23-11720,3100564E9989235E,ldsproule@icloud.com,English
3,R9O1129,ORSCRD,BB,110,1,110,Downchild Blues Band,2019-11-29,20:00:00.000,FRI,...,EZ9OK29,SOLD,Adult,48080,2280462.0,"Pope, Jane",17-59441,310056A6BA133548,pope-jane@hotmail.com,English
4,D9O0515,ORSCRD,F,126,1,126,Alberta Ballet,2019-05-15,20:00:00.000,WED,...,EZ9OE15,SOLD,Adult,48080,2280463.0,"Pope, Jane",36-23279,310054B7E2EE1B96,pope-jane@hotmail.com,English


In [21]:
gift_cards.head()

,package_id,package_title_en,package_title_fr,package_description_en,package_description_fr,package_image_url,package_url_en,package_url_fr
0,19489,Beethoven À la carte!,Beethoven À la carte!,3 Focus Festival concerts starting at $75,3 concerts du festival Focus à partir de 75 $,https://naccnaca-packages.s3.amazonaws.com/194...,https://nac-cna.ca/en/orchestra/packages/beeth...,https://nac-cna.ca/fr/orchestra/packages/beeth...
1,19454,Create Your Own Family,À la carte famille,3 or more concerts starting at $29,3 concerts ou plus à partir de 29 $,https://naccnaca-packages.s3.amazonaws.com/194...,https://nac-cna.ca/en/orchestra/packages/creat...,https://nac-cna.ca/fr/orchestra/packages/creat...
2,18923,Create Your Own,Forfait à la carte,NaN,NaN,https://naccnaca-packages.s3.amazonaws.com/189...,https://nac-cna.ca/en/dance/packages/create-yo...,https://nac-cna.ca/fr/dance/packages/create-yo...
3,18879,Music for a Sunday Afternoon,Musique pour un dimanche après-midi,4 concerts,4 concerts,https://naccnaca-packages.s3.amazonaws.com/188...,https://nac-cna.ca/en/orchestra/packages/music...,https://nac-cna.ca/fr/orchestra/packages/music...
4,18874,Create Your Own,À la carte,NaN,NaN,https://naccnaca-packages.s3.amazonaws.com/188...,https://nac-cna.ca/en/orchestra/packages/creat...,https://nac-cna.ca/fr/orchestra/packages/creat...


ds.productId
added_events.url(arena?, event_day, minor category, major category)
index_events.event_id(event_category)
?tickets_and_plans.event_name+email(ticket_type)